In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string
from nltk.util import ngrams
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression


In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string
from nltk.util import ngrams
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [8]:
import re

In [9]:
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [10]:
sample_submission

,sell_id,price
0,1100575026,0
1,1100549428,0
2,1100658222,0
3,1100937408,0
4,1101037972,0
...,...,...
34681,1101369263,0
34682,1101369581,0
34683,1101364889,0
34684,1101362518,0


In [14]:
RANDOM_SEED = 42
VAL_SIZE   = 0.2

In [12]:
data = pd.read_csv('/content/data_datasetcar.csv', low_memory=False) 

In [13]:
train = pd.read_csv('/content/train150121.csv', low_memory=False) 

In [15]:
y = train['price']

In [16]:
data.head()

,Unnamed: 0,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,bodyType2,bT_length,Nalog,years_old,age_intervals,sample
0,0,8,14,1.2,105.0,0,74000,2013,710,5,2014,3,2,2,1,1,0,4,7,2625.0,7,1,0
1,1,8,17,1.6,110.0,0,60563,2017,710,5,2017,2,2,0,1,1,0,4,7,2750.0,4,1,0
2,2,8,13,1.8,152.0,0,88000,2013,954,5,2014,3,2,0,1,1,0,4,7,6840.0,7,1,0
3,3,8,7,1.6,110.0,0,95000,2013,710,5,2014,0,2,0,1,1,0,4,7,2750.0,7,1,0
4,4,8,1,1.8,152.0,0,58536,2008,710,5,2012,0,2,0,1,1,0,4,7,6840.0,9,2,0


In [17]:
data.isna().sum()

Unnamed: 0             0
brand                  0
color                  0
engineDisplacement     0
enginePower            0
fuelType               0
mileage                0
modelDate              0
model_name             0
numberOfDoors          0
productionDate         0
vehicleTransmission    0
vendor                 0
Владельцы              0
ПТС                    0
Привод                 0
Руль                   0
bodyType2              0
bT_length              0
Nalog                  0
years_old              0
age_intervals          0
sample                 0
dtype: int64

In [18]:
columns_drop = ['Unnamed: 0']
data = data.drop(columns_drop, axis=1)

In [19]:
for column in ['bodyType2', 'brand', 'color', 'fuelType', 'model_name', 'vendor', 'Владельцы', 'vehicleTransmission', 'Привод', 'age_intervals']:
    data[column] = data[column].astype('category').cat.codes

In [20]:
bin_cols = ['ПТС', 'Руль']
label_encoder = LabelEncoder()
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
data.head()

,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,bodyType2,bT_length,Nalog,years_old,age_intervals,sample
0,8,14,1.2,105.0,0,74000,2013,710,5,2014,3,2,2,1,1,0,4,7,2625.0,7,1,0
1,8,17,1.6,110.0,0,60563,2017,710,5,2017,2,2,0,1,1,0,4,7,2750.0,4,1,0
2,8,13,1.8,152.0,0,88000,2013,954,5,2014,3,2,0,1,1,0,4,7,6840.0,7,1,0
3,8,7,1.6,110.0,0,95000,2013,710,5,2014,0,2,0,1,1,0,4,7,2750.0,7,1,0
4,8,1,1.8,152.0,0,58536,2008,710,5,2012,0,2,0,1,1,0,4,7,6840.0,9,2,0


In [21]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [22]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, random_state=RANDOM_SEED)

In [24]:
np.log(y_train)

34318     12.785491
42375     12.206073
178827    15.287983
21807     13.975075
5154      13.304685
            ...    
119879    13.763164
103694    12.254863
131932    16.077065
146867    13.440090
121958    12.206073
Name: price, Length: 151560, dtype: float64

In [ ]:
The best Score Xgboost parameters

In [25]:
import xgboost as xgb

In [26]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.02 ,max_depth=22, alpha=1,
                          n_estimators=15000,tree_method='gpu_hist')

In [27]:
xg_reg.fit(X_train, np.log(y_train))

XGBRegressor(alpha=1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=22, min_child_weight=1, missing=None, n_estimators=15000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, tree_method='gpu_hist',
             verbosity=1)

In [28]:
predict_test = np.exp(xg_reg.predict(X_test)) 
predict_submission = np.exp(xg_reg.predict(X_sub))


In [29]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 10.58%


In [30]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_xgboost18fin.csv', index=False)


In [31]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.02 ,max_depth=24, alpha=1,
                          n_estimators=15000,tree_method='gpu_hist')

In [ ]:
xg_reg.fit(X_train, np.log(y_train))


In [ ]:
predict_test = np.exp(xg_reg.predict(X_test)) 
predict_submission = np.exp(xg_reg.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 10.72%


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_xgboost2156.csv', index=False)


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('/content/sample_data/submission_xg_fin.csv', index=False)


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('/content/sample_data/submission_xg_finfinal.csv', index=False)


In [ ]:
%cd

/root


In [ ]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=5000)
gb.fit(X_train, np.log(y_train))

In [ ]:
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_1503gb.csv', index=False)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor


In [ ]:
import xgboost as xgb

In [ ]:
estimators = [
    ('xg_reg', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,learning_rate=0.007, max_depth=17, alpha=1,n_estimators=5000)),
    ('gb', GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=17, n_estimators=5000))
]

In [ ]:
reg5 = StackingRegressor(estimators=estimators)
reg5.fit(X_train, np.log(y_train))


NameError: ignored

In [ ]:
predict_test = np.exp(reg5.predict(X_test))
predict_submission = np.exp(reg5.predict(X_sub))


NameError: ignored

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_1504st.csv', index=False)
            

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor


In [ ]:
estimators = [
    ('xg_reg', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,learning_rate=0.007, max_depth=18, alpha=1,n_estimators=5000)),
    ('gb', GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=18, n_estimators=5000)
)
]

In [ ]:
reg4 = StackingRegressor(estimators=estimators)
reg4.fit(X_train, np.log(y_train))


In [ ]:
predict_test = np.exp(reg4.predict(X_test))
predict_submission = np.exp(reg4.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_catxgb.csv', index=False)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
X_train.shape

(151560, 21)

In [ ]:
model = Sequential()
model.add(Dense(4737, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mape', metrics=['mse', 'mape'])

In [ ]:
model.fit(X_train, np.log(y_train), epochs=5, validation_data=(X_test, np.log(y_test)))

Epoch 1/5
1521/4737 [========>.....................] - ETA: 6:30 - loss: 5092.0503 - mse: 60962710.2209 - mape: 5092.0503

KeyboardInterrupt: ignored

In [ ]:
preds = np.exp(model.predict(X_test))
preds_submission = np.exp(model.predict(X_sub))

In [ ]:
mape(y_test, preds)

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, preds))*100:0.2f}%")

Exception: Data must be 1-dimensional